Author : Muhammad Khairunnas

Assignment : GraphQL dan WebScrapping menggunakan beautifulsoup

In [42]:
# import library
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


# GraphQL

In [43]:
# mengambil data negara yang diawali huruf I

graphql_url = 'https://countries.trevorblades.com'
graphql_query = {
    'query': """
     {
        countries(filter: { name: { regex: "^I" } }) {
          code
          name
          currency
        }
     }
    """
}

graphql_response = requests.post(graphql_url, json=graphql_query)
graphql_response_data = graphql_response.json()

In [44]:
# mengambil data awsRegion dan provinsi dari negara Indonesia
graphql_url_2 = 'https://countries.trevorblades.com'
graphql_query_2 = {
    "query" : """
    {
      country(code: "ID"){
        code
        name
        awsRegion
        states {
          code
          name
        }
        phones
      }
    }

    """
}

graphql_response_2 = requests.post(graphql_url_2, json=graphql_query_2)


In [45]:
# menyimpan data provinsi untuk negara Indonesia kedalam file csv

country = None
if graphql_response_2.status_code == 200:
    country = graphql_response_2.json().get('data').get('country')

data_regions = []

if country:
    for state in country.get('states'):
        data_state = {
            "country_code": country.get('code'),
            "country_name": country.get('name'),
            "country_awsRegion": country.get('awsRegion'),
            "country_phone_code": country.get('phones'),
            "state_code": state.get('code'),
            "state_name": state.get('name'),
            }
        data_regions.append(data_state)
data_regions

[{'country_code': 'ID',
  'country_name': 'Indonesia',
  'country_awsRegion': 'ap-southeast-1',
  'country_phone_code': ['62'],
  'state_code': 'ID-AC',
  'state_name': 'Special Region of Aceh'},
 {'country_code': 'ID',
  'country_name': 'Indonesia',
  'country_awsRegion': 'ap-southeast-1',
  'country_phone_code': ['62'],
  'state_code': 'ID-BA',
  'state_name': 'Bali'},
 {'country_code': 'ID',
  'country_name': 'Indonesia',
  'country_awsRegion': 'ap-southeast-1',
  'country_phone_code': ['62'],
  'state_code': 'ID-BB',
  'state_name': 'Bangka–Belitung Islands'},
 {'country_code': 'ID',
  'country_name': 'Indonesia',
  'country_awsRegion': 'ap-southeast-1',
  'country_phone_code': ['62'],
  'state_code': 'ID-BT',
  'state_name': 'Banten'},
 {'country_code': 'ID',
  'country_name': 'Indonesia',
  'country_awsRegion': 'ap-southeast-1',
  'country_phone_code': ['62'],
  'state_code': 'ID-BE',
  'state_name': 'Bengkulu'},
 {'country_code': 'ID',
  'country_name': 'Indonesia',
  'country_a

In [46]:
# ubah data provinsi negara indonesia kedalam bentuk data frame

region_df = pd.DataFrame(data=data_regions)
region_df.head()

,country_code,country_name,country_awsRegion,country_phone_code,state_code,state_name
0,ID,Indonesia,ap-southeast-1,[62],ID-AC,Special Region of Aceh
1,ID,Indonesia,ap-southeast-1,[62],ID-BA,Bali
2,ID,Indonesia,ap-southeast-1,[62],ID-BB,Bangka–Belitung Islands
3,ID,Indonesia,ap-southeast-1,[62],ID-BT,Banten
4,ID,Indonesia,ap-southeast-1,[62],ID-BE,Bengkulu


In [47]:
# save dataframe provinsi indonesia kedalam csv

region_df.to_csv('indonesia_region_dataset.csv')

# BeautifulSoup

In [73]:
# get html dari halaman ruparupa.com
import re
import time

count_scrap_item = 3
product_list = []

for i in range(1, count_scrap_item+1):
    rupa_rupa_url = f'https://www.dekoruma.com/Sofa?gad_source=1&gclid=Cj0KCQjwlvW2BhDyARIsADnIe-KJvQLbTRZXcnVqvreGn9hheGcE7xdRQpnd82SxSjp6hKoRIlHXde8aAudGEALw_wcB&page={i}'
    rupa_rupa_response = requests.get(rupa_rupa_url)
    rupa_rupa_content = rupa_rupa_response.content
    soup = BeautifulSoup(rupa_rupa_content, 'html.parser')
    products = soup.find_all('li', class_='pure-u-1-4 phs mbxl')

    for product in products:
        if product.find("div", class_='css-901oao r-1umadx r-a023e6 r-b88u0q r-rjixqe'):
            product_after_discount = product.find("div", class_='css-901oao r-1umadx r-a023e6 r-b88u0q r-rjixqe').text
        elif product.find("div", class_='css-901oao r-1umadx r-1i10wst r-b88u0q r-135wba7'):
            product_after_discount = product_after_discount = product.find("div", class_='css-901oao r-1umadx r-1i10wst r-b88u0q r-135wba7').text
        else:
            product_after_discount = None

        data_product = {
            "product_image" : product.find("img", class_='css-9pa8cd').get("src"),
            "product_name" : product.find("div", class_='css-901oao css-cens5h r-1umadx r-1enofrn r-8ay8q1 r-1cwl3u0 r-15zivkp').text if product.find("div", class_='css-901oao css-cens5h r-1umadx r-1enofrn r-8ay8q1 r-1cwl3u0 r-15zivkp') else None,
            "product_price" : product.find("div", class_='css-901oao r-1umadx r-1enofrn r-8ay8q1 r-1cwl3u0 r-142tt33').text if product.find("div", class_='css-901oao r-1umadx r-1enofrn r-8ay8q1 r-1cwl3u0 r-142tt33') else None,
            "product_discount" : product.find("div", class_='css-901oao r-1umadx r-10x49cs r-b88u0q r-3ucsff r-56xrmm').text if product.find("div", class_='css-901oao r-1umadx r-10x49cs r-b88u0q r-3ucsff r-56xrmm') else None,
            "product_after_discount" : product_after_discount,
            "product_rating": re.findall(r'\d+', product.find("div", class_='css-901oao r-1umadx r-1gkfh8e r-8ay8q1 r-1f529hi').text if product.find("div", class_='css-901oao r-1umadx r-1gkfh8e r-8ay8q1 r-1f529hi') else ''),
            "service": product.find("div", class_='css-901oao r-1umadx r-1enofrn r-b88u0q r-8ay8q1 r-1cwl3u0').text if product.find("div", class_='css-901oao r-1umadx r-1enofrn r-b88u0q r-8ay8q1 r-1cwl3u0') else None,

        }
        product_list.append(data_product)
    time.sleep(20)

In [74]:
furniture_df = pd.DataFrame(data=product_list)
furniture_df.head()

,product_image,product_name,product_price,product_discount,product_after_discount,product_rating,service
0,https://media.dekoruma.com/catalogue/NRA-41515...,Heim Studio NOBU Sofa Bed Cream,"Rp2,423,000",45%,"Rp1,329,000",[2348],se-Jabodetabek
1,https://media.dekoruma.com/catalogue/NRA-41515...,Heim Studio NOBU Sofa Bed Abu,"Rp2,423,000",45%,"Rp1,329,000",[2330],se-Jabodetabek
2,https://media.dekoruma.com/catalogue/NRA-40684...,Heim Studio HATO Sofabed Cokelat,"Rp5,570,000",40%,"Rp3,329,000",[1047],se-Jabodetabek
3,https://media.dekoruma.com/catalogue/NRA-37970...,Heim Studio HIRO Sofa 2 Seater Abu-abu,"Rp5,444,000",44%,"Rp2,999,000",[967],se-Jabodetabek
4,https://media.dekoruma.com/catalogue/NRA-38021...,Heim Studio HIRO Sofa 3 Seater Abu-abu,"Rp7,010,000",32%,"Rp4,699,000",[1160],se-Jabodetabek


In [75]:
furniture_df.to_csv('furniture_dataset.csv')